In [1]:
!pip install tensorflow
!pip install matplotlib

In [2]:
import tensorflow as tf
from keras import models, layers 
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Dropout


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
NUM_CHANNELS = 3 #rgb

In [4]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory="/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres",
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle =True
)

Found 1856 files belonging to 2 classes.


In [5]:
def split_dataset(ds):
    train_size = int(len(ds)*0.8)
    val_size = int(len(ds)*0.1)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).take(val_size)
    return train_ds, val_ds, test_ds


train_ds, val_ds, test_ds = split_dataset(ds=dataset)

In [6]:
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [7]:
resize_and_rescale_layer = tf.keras.Sequential(
    layers=[
        layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
        layers.Rescaling(1./255)
    ]
)

In [8]:
data_augmentation_layer = tf.keras.Sequential(
    layers=[
        layers.RandomFlip(mode='horizontal_and_vertical'),
        layers.RandomContrast(factor=(0.0, 1)),
        layers.RandomRotation(factor=(-0.2, 0.3),fill_mode='reflect',interpolation='bilinear')
    ]
)

In [9]:
input_shape = (BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,NUM_CHANNELS)
model = models.Sequential(
    [
        resize_and_rescale_layer,
        data_augmentation_layer,
        
        layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, ),
        layers.MaxPooling2D((2, 2)),
        
        
        layers.Conv2D(filters = 64, kernel_size=(3, 3), activation='relu',),
        layers.MaxPooling2D((2, 2)),
        
        
        layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu',),
        layers.MaxPooling2D((2, 2)),
        
        
         layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu',),
        layers.MaxPooling2D((2, 2)),
        
        
        layers.Flatten(),
#         layers.Dense(32, activation='relu'),
        layers.Dense(2, activation='softmax')
    ]
)

In [10]:
model.build(input_shape=input_shape)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 256, 256, 3)         0         
                                                                 
 sequential_1 (Sequential)   (32, 256, 256, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (32, 127, 127, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (32, 62, 62, 64)         0         
 2D)                                                  

In [14]:

# earlystopping=tf.keras.callbacks.EarlyStopping(
#     monitor='Accuracy',
#     min_delta=0.0001,
#     patience=2,
#     restore_best_weights=True,
#     start_from_epoch=20
# )

# tensorboard= tf.keras.callbacks.TensorBoard()

chkpt = tf.keras.callbacks.ModelCheckpoint(
    'TyreCLF.ckpt',
    monitor = 'Accuracy',
    save_best_only = True,
    
    mode = 'auto',
    save_freq='epoch',
)


model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )


In [ ]:
history = model.fit(
    train_ds,
    epochs=100,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_ds,
    callbacks=[earlystopping,chkpt,tensorboard]
)

Epoch 1/100
46/46 [==============================] - 3s 60ms/step - loss: 0.1901 - accuracy: 0.9178 - val_loss: 0.2583 - val_accuracy: 0.8938
Epoch 2/100
46/46 [==============================] - 3s 59ms/step - loss: 0.1918 - accuracy: 0.9246 - val_loss: 0.2126 - val_accuracy: 0.9250
Epoch 3/100
46/46 [==============================] - 3s 60ms/step - loss: 0.1485 - accuracy: 0.9389 - val_loss: 0.3301 - val_accuracy: 0.8813
Epoch 4/100
46/46 [==============================] - 3s 60ms/step - loss: 0.1658 - accuracy: 0.9287 - val_loss: 0.2530 - val_accuracy: 0.9000
Epoch 5/100
24/46 [==============>...............] - ETA: 1s - loss: 0.1828 - accuracy: 0.9284

In [16]:
model.evaluate(test_ds)

5/5 [==============================] - 60s 808ms/step - loss: 0.4042 - accuracy: 0.8562


[0.40420326590538025, 0.856249988079071]

In [ ]:
class_names = dataset.class_names
class_names

In [ ]:
import numpy as np

def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence


plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")

In [ ]:
model.save('/kaggle/working/TyreCLF_118k_90')

In [ ]:
import shutil
shutil.make_archive("/kaggle/working/TyreCLF_118k_90", 'zip')
